In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44779
2,Huelva,Confirmados PDIA 14 días,712
3,Huelva,Tasa PDIA 14 días,"138,74544497924666"
4,Huelva,Confirmados PDIA 7 días,253
5,Huelva,Tasa PDIA 7 dias,"49,30140109515365"
6,Huelva,Total Confirmados,44978
7,Huelva,Curados,41160
8,Huelva,Fallecidos,406


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44779.0


/tmp/ipykernel_38470/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12405.0


/tmp/ipykernel_38470/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_38470/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_38470/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_38470/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2935 personas en los últimos 7 días 

Un positivo PCR cada 953 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44779.0,253.0,712.0,513170.0,49.301401,138.745445,120.0
Huelva-Costa,26480.0,180.0,453.0,289548.0,62.165859,156.450744,88.0
Huelva (capital),12405.0,49.0,151.0,143837.0,34.066339,104.979943,24.0
Cartaya,1901.0,42.0,71.0,20083.0,209.132102,353.532839,21.0
Condado-Campiña,13917.0,48.0,181.0,156231.0,30.723736,115.854088,19.0
Isla Cristina,3080.0,27.0,72.0,21393.0,126.209508,336.558687,14.0
Ayamonte,2015.0,22.0,47.0,21104.0,104.245641,222.706596,13.0
Sierra de Huelva-Andévalo Central,3963.0,22.0,70.0,67391.0,32.645309,103.871437,12.0
Almonte,2253.0,13.0,35.0,24507.0,53.046068,142.816338,9.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,32.0,2.0,6.0,396.0,505.050505,1515.151515,1.0
San Silvestre de Guzmán,24.0,3.0,7.0,614.0,488.599349,1140.065147,0.0
Villablanca,284.0,9.0,16.0,2885.0,311.958406,554.592721,3.0
Higuera de la Sierra,64.0,2.0,6.0,1291.0,154.918668,464.756003,2.0
San Bartolomé de la Torre,376.0,2.0,17.0,3761.0,53.177346,452.007445,0.0
Cerro de Andévalo (El),195.0,1.0,10.0,2327.0,42.973786,429.737860,0.0
Aroche,126.0,7.0,13.0,3054.0,229.207597,425.671251,5.0
Cartaya,1901.0,42.0,71.0,20083.0,209.132102,353.532839,21.0
Isla Cristina,3080.0,27.0,72.0,21393.0,126.209508,336.558687,14.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,1609.0,1.0,12.0,21474.0,4.656794,55.881531,1.0,0.083333
Cerro de Andévalo (El),195.0,1.0,10.0,2327.0,42.973786,429.737860,0.0,0.100000
Gibraleón,924.0,2.0,17.0,12737.0,15.702285,133.469420,1.0,0.117647
San Bartolomé de la Torre,376.0,2.0,17.0,3761.0,53.177346,452.007445,0.0,0.117647
Palma del Condado (La),1414.0,2.0,15.0,10801.0,18.516804,138.876030,2.0,0.133333
Trigueros,541.0,1.0,6.0,7862.0,12.719410,76.316459,0.0,0.166667
Beas,358.0,1.0,6.0,4341.0,23.036167,138.217001,0.0,0.166667
Rociana del Condado,660.0,1.0,5.0,7939.0,12.596045,62.980224,1.0,0.200000
Moguer,1927.0,7.0,30.0,21867.0,32.011707,137.193031,3.0,0.233333
